<a href="https://colab.research.google.com/github/Goshhh10/Movie-Recommendation-Engine/blob/main/Movie_Recommendation_Engine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import Libraries

In [ ]:
import numpy as np
import os
import pandas as pd
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer


Check the current working directory

In [ ]:
display(os.getcwd())

'G:\\acmegrade'

Change the current working directory and read data

In [ ]:
os.chdir('G:\\acmegrade\\')
movies_data = pd.read_csv('movies.csv')
display(movies_data.head())

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


Display the shape

In [ ]:
display(movies_data.shape)

(4803, 24)

Selecting the relevant features for recommendation

In [ ]:
selected_features = ['genres', 'keywords', 'tagline', 'cast','original_title', 'spoken_languages', 'release_date']
print(selected_features)

['genres', 'keywords', 'tagline', 'cast', 'original_title', 'spoken_languages', 'release_date']


Display the info

In [ ]:
display(movies_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   index                 4803 non-null   int64  
 1   budget                4803 non-null   int64  
 2   genres                4775 non-null   object 
 3   homepage              1712 non-null   object 
 4   id                    4803 non-null   int64  
 5   keywords              4391 non-null   object 
 6   original_language     4803 non-null   object 
 7   original_title        4803 non-null   object 
 8   overview              4800 non-null   object 
 9   popularity            4803 non-null   float64
 10  production_companies  4803 non-null   object 
 11  production_countries  4803 non-null   object 
 12  release_date          4802 non-null   object 
 13  revenue               4803 non-null   int64  
 14  runtime               4801 non-null   float64
 15  spoken_languages     

None

Check for Null Values

In [ ]:
display(movies_data.isna().sum())

index                      0
budget                     0
genres                    28
homepage                3091
id                         0
keywords                 412
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                  844
title                      0
vote_average               0
vote_count                 0
cast                      43
crew                       0
director                  30
dtype: int64

Display the selected columns

In [ ]:
display(movies_data[selected_features].head())


,genres,keywords,tagline,cast,original_title,spoken_languages,release_date
0,Action Adventure Fantasy Science Fiction,culture clash future space war space colony so...,Enter the World of Pandora.,Sam Worthington Zoe Saldana Sigourney Weaver S...,Avatar,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",2009-12-10
1,Adventure Fantasy Action,ocean drug abuse exotic island east india trad...,"At the end of the world, the adventure begins.",Johnny Depp Orlando Bloom Keira Knightley Stel...,Pirates of the Caribbean: At World's End,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",2007-05-19
2,Action Adventure Crime,spy based on novel secret agent sequel mi6,A Plan No One Escapes,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,Spectre,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",2015-10-26
3,Action Crime Drama Thriller,dc comics crime fighter terrorist secret ident...,The Legend Ends,Christian Bale Michael Caine Gary Oldman Anne ...,The Dark Knight Rises,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",2012-07-16
4,Action Adventure Science Fiction,based on novel mars medallion space travel pri...,"Lost in our world, found in another.",Taylor Kitsch Lynn Collins Samantha Morton Wil...,John Carter,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",2012-03-07


Check Null Values in selected columns

In [ ]:
display(movies_data[selected_features].isna().sum())

genres               28
keywords            412
tagline             844
cast                 43
original_title        0
spoken_languages      0
release_date          1
dtype: int64

Replacing the null values with null string

In [ ]:
for feature in selected_features:
    movies_data[feature] = movies_data[feature].fillna('')


Combining all the 7 selected features


In [ ]:
combined_features = movies_data['genres'] + ' ' + movies_data['keywords'] + ' ' + movies_data['tagline'] + ' ' + \
                    movies_data['cast'] + ' ' + movies_data['original_title'] + ' ' + \
                    movies_data['spoken_languages'] + ' ' + movies_data['release_date']


display combined list

In [ ]:
display(combined_features.head())

0    Action Adventure Fantasy Science Fiction cultu...
1    Adventure Fantasy Action ocean drug abuse exot...
2    Action Adventure Crime spy based on novel secr...
3    Action Crime Drama Thriller dc comics crime fi...
4    Action Adventure Science Fiction based on nove...
dtype: object

Converting the text data to feature vectors

In [ ]:
vectorizer = TfidfVectorizer()
feature_vectors = vectorizer.fit_transform(combined_features)

Display shape of the feature vector

In [ ]:
display(feature_vectors.shape)
print(feature_vectors)

(4803, 18255)

  (0, 11)	0.09241313016102944
  (0, 19)	0.09329203211709576
  (0, 137)	0.12102643547938367
  (0, 16643)	0.11044579035042021
  (0, 5366)	0.11044579035042021
  (0, 5355)	0.11044579035042021
  (0, 5273)	0.03265741210921921
  (0, 11196)	0.062219563529807634
  (0, 5245)	0.032650599638265936
  (0, 8127)	0.06223251855827844
  (0, 1198)	0.2684772429113252
  (0, 13586)	0.20306793795683545
  (0, 10626)	0.15112764341933999
  (0, 9184)	0.2261063329974745
  (0, 15226)	0.1565035922743859
  (0, 17500)	0.18674416031437607
  (0, 14629)	0.19382897980334365
  (0, 13879)	0.2049208780405492
  (0, 18123)	0.1889282428659192
  (0, 17848)	0.22250639453336296
  (0, 13906)	0.15015727141618013
  (0, 11962)	0.24729178795439982
  (0, 11632)	0.07423475273781778
  (0, 17838)	0.11810220393495416
  (0, 15918)	0.05296973812432831
  :	:
  (4802, 5730)	0.23664028160410744
  (4802, 6811)	0.22857894994593891
  (4802, 2620)	0.24728524449010963
  (4802, 3912)	0.2629960705873078
  (4802, 4932)	0.25690160217403807
  (4802, 4664

similarity scores using cosine similarity

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(feature_vectors)
display(similarity.shape)

(4803, 4803)

Function to find close match

In [ ]:
import difflib

def find_close_match(movie_name, list_of_titles):
    matches = difflib.get_close_matches(movie_name, list_of_titles, n=1)
    return matches[0] if matches else None

movie_name = input('Enter your favorite movie name: ')
list_of_titles = movies_data['title'].tolist()
close_match = find_close_match(movie_name, list_of_titles)
print("Close Match:", close_match)


Enter your favorite movie name:  spider man


Close Match: Spider-Man


Function to get movie index

In [ ]:
def get_movie_index(movie_name, movies_data):
    return movies_data[movies_data.title == movie_name].index[0]

if close_match:
    movie_index = get_movie_index(close_match, movies_data)
    print("Movie Index:", movie_index)


Movie Index: 159


Function to get similarity scores


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

def get_similarity_scores(movie_index, similarity_matrix):
    return list(enumerate(similarity_matrix[movie_index]))

if close_match:
    similarity_scores = get_similarity_scores(movie_index, similarity)
    print("Similarity Scores:", similarity_scores[:5])

Similarity Scores: [(0, 0.026480508423507278), (1, 0.03725184111101096), (2, 0.015847635006070025), (3, 0.009899941600862777), (4, 0.07972370993354826)]


Function to sort and get top movies

In [ ]:
def get_top_movies(similarity_scores, top_n=30):
    sorted_movies = sorted(similarity_scores, key=lambda x: x[1], reverse=True)[:top_n]
    return sorted_movies

if close_match:
    top_movies = get_top_movies(similarity_scores)
    print("Top Movies:", top_movies)

Top Movies: [(159, 1.0), (5, 0.36179421705577236), (30, 0.30825889792678013), (2601, 0.197227632328879), (3575, 0.16172704202565097), (382, 0.15052786656970557), (20, 0.14744750322169378), (1559, 0.14651366436231106), (49, 0.13973012895114978), (1193, 0.13587630356443403), (2361, 0.13394273093376552), (38, 0.1261760222409854), (328, 0.12070208767259631), (37, 0.11641304966019793), (1364, 0.11575086693601504), (3077, 0.11547989129371401), (1793, 0.11124060589110697), (677, 0.10869515479129192), (1796, 0.10515156096077885), (431, 0.10461763692491319), (787, 0.10428506398325069), (3085, 0.10371691660492588), (3177, 0.10361796581147892), (3155, 0.10331019775105418), (1523, 0.10269746367344407), (978, 0.10036284403926092), (2588, 0.09894919631747906), (4441, 0.09706833763663343), (2529, 0.0968996750415458), (4427, 0.09615911693910041)]


Function to display movie recommendations

In [ ]:
def display_recommendations(top_movies, movies_data):
    recommendations = pd.DataFrame(
        [(i+1, movies_data.iloc[movie[0]]['title'],
          movies_data.iloc[movie[0]]['original_title'],
          movies_data.iloc[movie[0]]['spoken_languages'],
          movies_data.iloc[movie[0]]['release_date'])
         for i, movie in enumerate(top_movies)],
        columns=["Rank", "Title", "Original Title", "Language", "Release Date"]
    )
    display(recommendations)

if close_match:
    display_recommendations(top_movies, movies_data)

,Rank,Title,Original Title,Language,Release Date
0,1,Spider-Man,Spider-Man,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",2002-05-01
1,2,Spider-Man 3,Spider-Man 3,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",2007-05-01
2,3,Spider-Man 2,Spider-Man 2,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",2004-06-25
3,4,Barney's Great Adventure,Barney's Great Adventure,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",1998-01-01
4,5,Clerks II,Clerks II,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",2006-05-25
5,6,Seabiscuit,Seabiscuit,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",2003-07-22
6,7,The Amazing Spider-Man,The Amazing Spider-Man,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",2012-06-27
7,8,The Notebook,The Notebook,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",2004-06-25
8,9,The Great Gatsby,The Great Gatsby,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",2013-05-10
9,10,The Count of Monte Cristo,The Count of Monte Cristo,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",2002-01-23
